In this notebook, I will mainly add embeddings. <br>
Note: Suspect something wrong with output layer, also, dont think I used embedding properly, will be redoing this notebook!

In [11]:
%cd ..
%cd root
import numpy as np
import pandas as pd
df = pd.read_csv("dwdata_2.csv")
df.head()
df.fillna('', inplace=True)
print(len(df))
x_1 = df["X_1"].tolist()
y_1 = df["Y_1"].tolist()

/
/root
76543


In [6]:
word2vec_index = {}
f = open("glove.6B.100d.txt",encoding="utf8")
for line in f:
    words = line.split()
    word = words[0]
    index = np.asarray(words[1:], dtype="float32")
    word2vec_index[word] = index
f.close()

print("The number of word vectors are: ", len(word2vec_index))

The number of word vectors are:  95172


In [12]:
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.sequence import pad_sequences
token = tfds.features.text.SubwordTextEncoder.build_from_corpus(x_1+y_1, target_vocab_size=1e12)
# token_y = tfds.features.text.SubwordTextEncoder.build_from_corpus(y_1, target_vocab_size=1e12)

vocab_size = token.vocab_size + 2 # adding the start and end characters

inputs = [token.encode(sentence) for sentence in x_1]
outputs = [[vocab_size - 2] + token.encode(sentence) + [vocab_size - 1] for sentence in y_1]
print("{},{}".format(len(inputs),len(outputs)))
MAXLEN = 70
idx_to_shorten = [count for count, sent in enumerate(inputs) if len(sent) > MAXLEN]
for idx in idx_to_shorten:
    inputs[idx] = inputs[idx][:MAXLEN]
    outputs[idx] = outputs[idx][:MAXLEN]
idx_to_shorten = []
idx_to_shorten = [count for count, sent in enumerate(outputs) if len(sent) > MAXLEN]
for idx in idx_to_shorten:
    inputs[idx] = inputs[idx][:MAXLEN]
    outputs[idx] = outputs[idx][:MAXLEN]
# deleted min len because short convos are still good :)
inputs = pad_sequences(inputs, value=0, padding="post", maxlen=MAXLEN)
outputs = pad_sequences(outputs, value=0, padding="post", maxlen=MAXLEN)
print("{},{}".format(inputs.shape,outputs.shape))

76543,76543
(76543, 70),(76543, 70)


In [0]:
vocab = set()
for sentence in (x_1 + y_1):
    words = sentence.split(" ")
    for word in words:
        if word not in vocab:
            vocab.add(word)

In [14]:
embedding_matrix = np.zeros((vocab_size, 100))
count = 0
count_1 = 0
not_glove = []
for i, word in enumerate(token.subwords):
    if (word[-1] == "_"):
        word = word[:-1]
    embedding_vector = word2vec_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector
        count_1 += 1
    else:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(100)
        not_glove.append(word)
        count += 1
print("In GLOVE: {}, Not in GLOVE: {}".format(count_1, count))
# HALF OF THE WORDS ARE NOT IN GLOVE WHYYYYYYYYYYY (glove.6B.100d)

In GLOVE: 27940, Not in GLOVE: 34939


NameError: ignored

In [16]:
from scipy.ndimage.interpolation import shift
inputs_1 = inputs
inputs_2 = outputs
outputs_1 = outputs
outputs_1 = shift(outputs_1, [0,-1], cval=0)
print("{},{}".format(inputs_1.shape,outputs_1.shape))

(76543, 70),(76543, 70)


In [0]:
#https://towardsdatascience.com/how-to-implement-seq2seq-lstm-model-in-keras-shortcutnlp-6f355f3e5639
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Bidirectional, Dropout, TimeDistributed
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Model
latent_dim = 256
Shared_Embedding = Embedding(vocab_size,100,weights=[embedding_matrix],input_length=MAXLEN,trainable=True)

encoder_inputs = Input(shape=(MAXLEN,), dtype='int32')
encoder_embedding = Shared_Embedding(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(latent_dim, return_state = True)(encoder_embedding)

decoder_inputs = Input(shape=(MAXLEN,), dtype='int32')
decoder_embedding = Shared_Embedding(decoder_inputs)
decoder_outputs, _, _ = LSTM(latent_dim, return_state=True, return_sequences=True)(decoder_embedding, initial_state=[state_h, state_c])

outputs_model = Dense(1, activation='softmax')(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], outputs_model)

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.05), metrics=['accuracy'])

In [20]:
hist = model.fit([inputs_1, inputs_2], outputs_1,batch_size=32,epochs=100,validation_split=0.2, verbose = 1)
model.save("2nd_m.h5")

Epoch 1/100
1914/1914 [==============================] - 76s 40ms/step - loss: 386584.4062 - accuracy: 0.0065 - val_loss: 386322.4688 - val_accuracy: 0.0063
Epoch 2/100
1914/1914 [==============================] - 76s 40ms/step - loss: 386592.7812 - accuracy: 0.0065 - val_loss: 386322.4688 - val_accuracy: 0.0063
Epoch 3/100
1914/1914 [==============================] - 76s 40ms/step - loss: 386585.0938 - accuracy: 0.0065 - val_loss: 386322.4688 - val_accuracy: 0.0063
Epoch 4/100
1914/1914 [==============================] - 76s 40ms/step - loss: 386589.5000 - accuracy: 0.0065 - val_loss: 386322.4688 - val_accuracy: 0.0063
Epoch 5/100
1914/1914 [==============================] - 76s 40ms/step - loss: 386586.6562 - accuracy: 0.0065 - val_loss: 386322.4688 - val_accuracy: 0.0063
Epoch 6/100
1914/1914 [==============================] - 77s 40ms/step - loss: 386601.4375 - accuracy: 0.0065 - val_loss: 386322.4688 - val_accuracy: 0.0063
Epoch 7/100
1914/1914 [==============================] - 7

KeyboardInterrupt: ignored

In [0]:
# use bidirectional lstm next time?
#https://github.com/oswaldoludwig/Adversarial-Learning-for-Generative-Conversational-Agents !!!!GAANNN!!!!
# Put attention in as well!
encoder_model = Model(encoder_inputs, [state_h, state_c])
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = LSTM(latent_dim, return_sequences=True, return_state=True)(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)

In [0]:
def decode_sequence(input_seq):
    input_seq = np.expand_dims(input_seq,axis=-1)
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1,1,1))
    target_seq[0, 0, 0] = vocab_size_y - 2

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = output_tokens[:,-1,:]
        decoded_sentence.append(sampled_token_index[0][0])

        if (sampled_token_index == vocab_size_y - 1 or len(decoded_sentence) > MAXLEN):
            stop_condition = True

        target_seq = np.zeros((1,1,1))
        target_seq[0,0,0] = sampled_token_index
        states_value = [h, c]
    pred = token_y.decode([int(i) for i in decoded_sentence])
    return pred
index_try = 33
input_seq = inputs_1[index_try: index_try + 1]
pred = decode_sequence(input_seq)
print('-')
print('Input sentence:', token_x.decode(np.squeeze(inputs_1[index_try], axis=-1)))
print('Decoded sentence:', pred)